In [1]:
"""Libraries"""

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import random
from scipy.integrate import solve_ivp
from multiprocessing import Pool
import importlib

# my own file:
import diffeq as de
importlib.reload(de)

<module 'diffeq' from 'F:\\OneDrive - Budapesti Műszaki és Gazdaságtudományi Egyetem\\TDK\\diffeq.py'>

In [3]:
"""Spacing"""

def expspace(Range, num, order=1):
    num -= 1
    a = (Range[1] - Range[0]) / (np.exp(num * order) - 1.0)
    b = Range[0] - a * 1.0
    nums = np.linspace(0, num, num+1)
    return a * np.exp(nums * order) + b

In [34]:
"""Control parameters"""
# Equilibrium radius [um --> m]
R_E_list = [1e-6*x for x in np.linspace(0.1, 30.0, 20)]
# R_star / R_E [-]
ratio_list = np.linspace(1.5, 30.0, 20)
# Ambient pressure [Pa]
P_inf_list = [1e5*x for x in np.linspace(1.0, 100.0, 20)]
# Accommodation coeff. for evaporation [-]
alfa_M_list = [0.05, 0.15, 0.25, 0.35]
# Ambient temperature [°C --> K]
T_inf_list = [273.15+x for x in [5.0, 20.0, 35.0, 50.0]]
# Surface tension modifier [-]
surfactant_list = [0.25, 0.5, 0.75, 1.0]
# Index of neutral gas [Ar: 9; He: 10]
index_list = [9, 10]

print(f'R_E: {len(R_E_list)}')
print(f'ratio: {len(ratio_list)}')
print(f'P_inf: {len(P_inf_list)}')
print(f'alfa_M: {len(alfa_M_list)}')
print(f'T_inf: {len(T_inf_list)}')
print(f'surfactant: {len(surfactant_list)}')
print(f'index: {len(index_list)}')
print(f'total combinations: {len(R_E_list) * len(ratio_list) * len(P_inf_list) * len(alfa_M_list) * len(T_inf_list) * len(surfactant_list) *len(index_list)}')

parameters = []
ID = 0
for R_E in R_E_list:
    for ratio in ratio_list:
        for P_inf in P_inf_list:
            for alfa_M in alfa_M_list:
                for T_inf in T_inf_list:
                    for surfactant in surfactant_list:
                        for index in index_list:
                            P_v = de.VapourPressure(T_inf)
                            parameters.append([ID, R_E, ratio, P_inf, alfa_M, T_inf, P_v, index, surfactant])
                            ID += 1
                            
random.seed(0)
random.shuffle(parameters)

R_E: 20
ratio: 20
P_inf: 20
alfa_M: 4
T_inf: 4
surfactant: 4
index: 2
total combinations: 1024000


In [4]:
"""Create folder to save files"""

study_name = 'full study try 3'
file_format = '.csv'
file_base_name = 'output_'
seperator = ','
line_end = '\n'

parent_dir = os.getcwd()
all_files = os.listdir(parent_dir)
exists = False
for file in all_files:
    if os.path.isdir(file):
        if os.path.basename(file) == study_name:
            exists = True
            
save_dir = os.path.join(parent_dir, study_name)
if not exists:
    os.mkdir(save_dir)
    
def create_line(data, seperator=',', line_end='\n'):
    line = ''
    for d in data:
        line += str(d) + seperator
    line = line[:-1]
    line += line_end
    return line
    
def new_file(save_dir, file_base_name, number=0, file_format='.txt'):
    file = os.path.join(save_dir, file_base_name + str(number) + file_format)
    f = open(file, 'w')
    first_line = create_line(de.datalist, seperator)
    f.write(first_line)
    return f

In [ ]:
"""Parameter study, multithread"""
n = 800000
lines = 1
number = 41
max_lines = 20000

start = time.process_time()
file = new_file(save_dir, file_base_name, number, file_format)
with Pool() as pool:
    results = pool.imap_unordered(de.solve, parameters[800000:900000])

    for data in results:
        if lines > max_lines:
            lines = 1
            number += 1
            file.close()
            file = new_file(save_dir, file_base_name, number, file_format)
        lines += 1
        n += 1
        line = create_line(data, seperator, line_end)
        file.write(line)
        print(f'index: {n}/{len(parameters)},   error: {data[1]},   steps: {data[2]},   runtime: {data[3]} s   |   R_E={data[4]*1e6} [um]; ' + 
              f'ratio={data[5]}; P_inf={data[6]}; alfa_M={data[7]}; T_inf={data[8]} [K]; index={data[10]}; surfactant={data[11]}' +
              '                                        ', end='\r')
        
'R_E [m]', 'R_0/R_E [-]', 'P_inf [Pa]', 'alfa_M [-]', 'T_inf [K]', 'vapour pressure [Pa]', 'index [Ar: 9; He: 10]', 'surfactant [-]',
file.close()
end = time.process_time()
elapsed = end - start
print(f"\n{elapsed:.2f} s")